<a href="https://colab.research.google.com/github/Ananya29J/CodeHer_25-InSpec-AI/blob/main/Vehicle_Defect_Detection_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

***Mounting Drive***

In [2]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive






***Unzip Dataset File***

In [ ]:
import zipfile
import os

zip_path = '/content/drive/MyDrive/Vehicle_Defect_Detection (1)/Vehicle_Dataset/archive (1).zip'
extract_path = '/content/drive/MyDrive/Vehicle_Defect_Detection (1)/Vehicle_Dataset/'

# Extract the ZIP file
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

# Print the contents of the extracted folder
print("Contents of extracted folder:", os.listdir(extract_path))


Contents of extracted folder: ['archive (1).zip', 'car_parts_prices.json', 'archive (2).zip', 'annotation LABELME', 'data3a', 'yolo_labels']


***Model Definition (MNV2)***

In [ ]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from sklearn.utils.class_weight import compute_class_weight
import numpy as np
import os

# === 1. Set Paths ===
train_dir = "/content/drive/MyDrive/Vehicle_Defect_Detection (1)/Vehicle_Dataset/data3a/training"
val_dir = "/content/drive/MyDrive/Vehicle_Defect_Detection (1)/Vehicle_Dataset/data3a/validation"
img_size = (224, 224)
batch_size = 32

# === 2. Data Augmentation ===
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.3,
    horizontal_flip=True,
    brightness_range=[0.8, 1.2]
)

val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical'
)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical'
)

# === 3. Compute Class Weights ===
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(train_generator.classes),
    y=train_generator.classes
)
class_weights_dict = dict(enumerate(class_weights))

# === 4. Load Pretrained MobileNetV2 Base ===
base_model = MobileNetV2(input_shape=(224, 224, 3), include_top=False, weights='imagenet')
base_model.trainable = False  # Freeze base model initially

# === 5. Add Custom Layers ===
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)                     # Increased dropout
x = Dense(128, activation='relu')(x)    # New layer
x = Dropout(0.3)(x)                     # New dropout
output = Dense(3, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=output)

# === 6. Compile Model ===
model.compile(optimizer=Adam(learning_rate=1e-4),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# === 7. Callbacks ===
callbacks = [
    EarlyStopping(patience=3, restore_best_weights=True),
    ModelCheckpoint("best_model.keras", save_best_only=True, monitor='val_accuracy'),
    ReduceLROnPlateau(monitor='val_accuracy', factor=0.5, patience=2)
]

# === 8. Train Top Layers ===
initial_epochs = 15

steps_per_epoch = train_generator.samples // batch_size
validation_steps = val_generator.samples // batch_size

history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=initial_epochs,
    class_weight=class_weights_dict,
    callbacks=callbacks,
    steps_per_epoch=steps_per_epoch,
    validation_steps=validation_steps
)

# === 9. Fine-Tune Base Model ===
base_model.trainable = True
for layer in base_model.layers[:100]:
    layer.trainable = False  # Freeze first 100 layers

model.compile(optimizer=Adam(learning_rate=1e-5),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

fine_tune_epochs = 10

history_finetune = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=fine_tune_epochs,
    class_weight=class_weights_dict,
    callbacks=callbacks,
    steps_per_epoch=steps_per_epoch,
    validation_steps=validation_steps
)

# === 10. Final Evaluation and Save ===
loss, accuracy = model.evaluate(val_generator)
print(f"Validation Accuracy: {accuracy*100:.2f}%")

# Save with accuracy in filename
model.save(f"fine_tuned_vehicle_model_acc_{accuracy*100:.2f}.keras")


Found 1383 images belonging to 3 classes.
Found 248 images belonging to 3 classes.
9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/15
43/43 ━━━━━━━━━━━━━━━━━━━━ 331s 7s/step - accuracy: 0.3494 - loss: 1.3563 - val_accuracy: 0.5759 - val_loss: 0.9723 - learning_rate: 1.0000e-04
Epoch 2/15
 1/43 ━━━━━━━━━━━━━━━━━━━━ 54s 1s/step - accuracy: 0.2812 - loss: 1.2677

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/epoch_iterator.py:107: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


43/43 ━━━━━━━━━━━━━━━━━━━━ 14s 310ms/step - accuracy: 0.2812 - loss: 1.2677 - val_accuracy: 0.5580 - val_loss: 0.9707 - learning_rate: 1.0000e-04
Epoch 3/15
43/43 ━━━━━━━━━━━━━━━━━━━━ 202s 4s/step - accuracy: 0.4126 - loss: 1.1194 - val_accuracy: 0.6205 - val_loss: 0.8975 - learning_rate: 1.0000e-04
Epoch 4/15
43/43 ━━━━━━━━━━━━━━━━━━━━ 20s 441ms/step - accuracy: 0.3750 - loss: 1.2400 - val_accuracy: 0.5938 - val_loss: 0.9036 - learning_rate: 1.0000e-04
Epoch 5/15
43/43 ━━━━━━━━━━━━━━━━━━━━ 143s 3s/step - accuracy: 0.4798 - loss: 1.0431 - val_accuracy: 0.6295 - val_loss: 0.8758 - learning_rate: 1.0000e-04
Epoch 6/15
43/43 ━━━━━━━━━━━━━━━━━━━━ 13s 290ms/step - accuracy: 0.4062 - loss: 1.1241 - val_accuracy: 0.6518 - val_loss: 0.8634 - learning_rate: 1.0000e-04
Epoch 7/15
43/43 ━━━━━━━━━━━━━━━━━━━━ 142s 3s/step - accuracy: 0.5071 - loss: 1.0047 - val_accuracy: 0.6741 - val_loss: 0.8434 - learning_rate: 1.0000e-04
Epoch 8/15
43/43 ━━━━━━━━━━━━━━━━━━━━ 19s 421ms/step - accuracy: 0.4375 - l

***Evaluate and Save***

In [ ]:
# Evaluate model performance
loss, accuracy = model.evaluate(val_generator)
print(f"Validation Accuracy: {accuracy*100:.2f}%")

# Create folder path
save_path = "/content/drive/My Drive/Vehicle_Defect_Detection/models/"
os.makedirs(save_path, exist_ok=True)

# Save the model
model.save(save_path + "vehicle_damage_model.h5")

8/8 ━━━━━━━━━━━━━━━━━━━━ 11s 1s/step - accuracy: 0.6987 - loss: 0.7367


Validation Accuracy: 68.95%


***Load Model***

In [ ]:
from tensorflow.keras.models import load_model

# Load previously trained model
model = load_model("vehicle_damage_model.h5")


***Image Augmentation + Normalization***

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_dir = "/content/vehicle_data/data3a/training"
val_dir = "/content/vehicle_data/data3a/validation"

# Image augmentation and normalization
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode="nearest"
)

val_datagen = ImageDataGenerator(rescale=1./255)

# Load images again to refresh class count
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)


Found 1383 images belonging to 3 classes.
Found 248 images belonging to 3 classes.


***Car Parts Price File Load***

In [13]:
import json

# Define path
json_file_path = "/content/drive/My Drive/Vehicle_Defect_Detection/Vehicle_Dataset/car_parts_prices.json"

# Load JSON data
with open(json_file_path, 'r') as file:
    car_parts_prices = json.load(file)

# View a sample
print(list(car_parts_prices.items())[:5])


[('HONDA', {'City': {'Bonnet': 15000, 'Bumper': 10000, 'Dickey': 8000, 'Door': 20000, 'Fender': 5000, 'Light': 3000, 'Windshield': 8000}, 'Amaze': {'Bonnet': 12000, 'Bumper': 8000, 'Dickey': 6000, 'Door': 18000, 'Fender': 4000, 'Light': 2500, 'Windshield': 7000}, 'WR-V': {'Bonnet': 16000, 'Bumper': 11000, 'Dickey': 9000, 'Door': 22000, 'Fender': 6000, 'Light': 3500, 'Windshield': 9000}, 'Jazz': {'Bonnet': 14000, 'Bumper': 9000, 'Dickey': 7000, 'Door': 19000, 'Fender': 4500, 'Light': 2800, 'Windshield': 8000}, 'HR-V': {'Bonnet': 18000, 'Bumper': 12000, 'Dickey': 10000, 'Door': 24000, 'Fender': 7000, 'Light': 4000, 'Windshield': 10000}, 'Pilot': {'Bonnet': 22000, 'Bumper': 15000, 'Dickey': 13000, 'Door': 28000, 'Fender': 8000, 'Light': 5000, 'Windshield': 12000}, 'CR-V': {'Bonnet': 20000, 'Bumper': 13000, 'Dickey': 11000, 'Door': 26000, 'Fender': 7500, 'Light': 4500, 'Windshield': 11000}, 'Accord': {'Bonnet': 22000, 'Bumper': 15000, 'Dickey': 13000, 'Door': 28000, 'Fender': 8000, 'Light'

***Price Estimate Prediction***

In [23]:
import json
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing import image

# Load the trained MobileNetV2 model
model_path = '/content/drive/MyDrive/Vehicle_Defect_Detection/models/vehicle_damage_model.h5'
model = tf.keras.models.load_model(model_path)

# Load the price data
price_file_path = '/content/drive/My Drive/Vehicle_Defect_Detection/Vehicle_Dataset/car_parts_prices.json'
with open(price_file_path, 'r') as f:
    price_data = json.load(f)

# Define class labels used in training
class_labels = ['minor', 'moderate', 'severe']

# Define severity multipliers
severity_multiplier = {
    'minor': 0.5,
    'moderate': 1.0,
    'severe': 1.5}

# Function to predict severity
def predict_severity(image_path):
    img = image.load_img(image_path, target_size=(224, 224))
    img_array = image.img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)

    predictions = model.predict(img_array)
    severity_index = np.argmax(predictions)
    severity_label = class_labels[severity_index]

    return severity_label


# Function to estimate repair cost
def estimate_repair_cost(brand, model_name, part, severity):
    brand = brand.upper().strip()
    model_name = model_name.strip()
    part_input = part.strip().lower()
    severity = severity.lower().strip()

    try:

        available_parts = price_data[brand][model_name]
        matched_part = None
        for actual_part in available_parts.keys():
            if actual_part.strip().lower() == part_input:
                matched_part = actual_part
                break

        if not matched_part:
            return f"Unknown: part '{part}' not found"

        base_price = price_data[brand][model_name][matched_part]
        multiplier = severity_multiplier.get(severity, 1.0)
        estimated_cost = int(base_price * multiplier)
        return f"₹{estimated_cost}"
    except KeyError as e:
        return f"Unknown: {e.args[0]} not found"

